# Web Scraping for Indeed.com and Predicting Salaries

In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import re
import time
import random
import csv
import sys

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [4]:
URL = "https://www.indeed.com/jobs?q=data%20scientist%20%2420%2C000&l=New%20York&start=10%22&vjk=b9dedd85514851bc"

In [7]:
# Proxy setup

proxy = {
        'https': "stuartlee165%40outlook.com:Rockwood123@91.189.176.133:443/",
        'http': "stuartlee165%40outlook.com:Rockwood123@91.189.176.133:443/"
}

URL = "https://www.indeed.com/jobs?q=data%20scientist%20%2420%2C000&l=Holtsville&start=10%22&vjk=b9dedd85514851bc"
r = requests.get(URL, proxies = proxy)
soup = BeautifulSoup(r.text, 'html.parser')


import requests

response = requests.get(
    "https://httpbin.org/get",
    proxies={
        "http": "http://116df877246d49fc9dd96b8efd4787e3:@proxy.crawlera.com:8011/",
        "https": "http://116df877246d49fc9dd96b8efd4787e3:@proxy.crawlera.com:8011/",
    },
    verify='/path/to/zyte-proxy-ca.crt' 
)
print(response.text)


In [10]:
# My initial strategy was to get a list of all the USA cities and a list of data science job titles from some CSVs I find online (this turned out to be very inefficient). Below code is me cleaning these lists up. Dropping duplicates.
addresses = pd.read_csv('/Users/stuart/Desktop/GA2/DSI20-lessons/projects/project-4/us_cities_states_counties.csv', sep='|', index_col=False) 

In [11]:
cities = addresses.City
cities = cities.drop_duplicates()
cities.head(5)
cities.shape

(18857,)

In [12]:
job_titles = pd.read_excel('/Users/stuart/Desktop/GA2/DSI20-lessons/projects/project-4/jobs_summary.xlsx')

In [13]:
job_titles_dscience = job_titles[job_titles.Category == 'Data Science']['Job Title']

In [14]:
job_titles_dscience.shape

(35,)

In [15]:
job_titles_dscience.loc[195] = 'scientist data'
job_titles_dscience.loc[196] = 'principal data scientist'
job_titles_dscience.loc[199] = 'decision data scientist'
job_titles_dscience.loc[202] = 'staff data scientist'
job_titles_dscience.loc[204] = 'lead data scientist'
job_titles_dscience.loc[212] = 'positive bioscience data'
job_titles_dscience.loc[217] = 'chief data science officer'
job_titles_dscience.loc[218] = 'president chief data scientist'
job_titles_dscience.loc[205] = 'analytic data science scientist'


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [16]:
cities = cities.str.replace(' ', '%20')

In [17]:
job_titles_dscience = job_titles_dscience.str.replace(' ', '%20')

I needed a function to estimate how many pages to search for under each city and job title. This is called later in my main scrape code. 

E.g. Chicago - data scientist would have many more pages to loop through than Wyoming - data intern (probably). Basically this finds the 'jobs found = 1000' item on the Indeed page and based on the fact that most pages list about 18 jobs calculates that it should iterate though 1000/18 times. Regex is used to filter out the '1000' number from the other bits of text. 

In [18]:
## Function that estimates number of pages to iterate through per search result

def pages_estimate(url):
    try:
        r = requests.get(url, proxies=proxies, verify='/Users/stuart/Downloads/zyte-smartproxy-ca.crt')
        soup = BeautifulSoup(r.text, 'html.parser')

        # Return number of pages on search result:
        job_found_text = soup.find('div', attrs={'id':'searchCountPages'}).text
        pattern = re.compile(r'(?<=of.)\d+ |(?<=of.)\d,\d+') ## filter so only shows integer
        number_text = re.findall(pattern, job_found_text)
        number_text = number_text[0].replace(",","")
        jobs_found = int(number_text)
        pages_to_loop = (round(jobs_found /18)*10)+10 # Set number of pages to loop for
    except:
        pages_to_loop = 20
    return pages_to_loop


In [19]:
pages_estimate('https://www.indeed.com/jobs?q=data%20scientist&l=New%20York&start=0&vjk=b9dedd85514851bc')

1120

In [20]:
cities

0                                Holtsville
2                                  Adjuntas
6                                    Aguada
12                                Aguadilla
45                                  Maricao
47                                   Anasco
54                                  Angeles
55                                  Arecibo
123                                Bajadero
125                             Barceloneta
151                                Boqueron
153                             Cabo%20Rojo
198                                Penuelas
217                                   Camuy
224                                Castaner
226                                 Rosario
227                         Sabana%20Grande
241                                  Ciales
246                                  Utuado
258                                  Dorado
309                                Ensenada
311                                 Florida
318                             

In [21]:
cities2 = ['Chicago', 'New%20York']

In [27]:
cities3 = ['New%20York', 'Chicago', 'San%20Francisco', 'Austin', 'Seattle', 
    'Los%20Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami']

In [52]:
cities4= ['nashville', 'washington', 'Oklahoma', 'Seattle', 'Texas']

In [48]:
cities5= ['london']

In [53]:
job_titles_dscience2 = ['data%20science', 'data%20engineer']

In [25]:
job_titles_dscience

187                         data%20scientist
188                senior%20data%20scientist
189                 chief%20data%20scientist
190                        chief%20scientist
191             principal%20data%20scientist
192                  lead%20data%20scientist
193                       senior%20scientist
194                director%20data%20science
195                         scientist%20data
196             principal%20data%20scientist
197                  data%20science%20intern
198                       datasciencecentral
199              decision%20data%20scientist
200                 sr%20%20data%20scientist
201                 staff%20data%20scientist
202                 staff%20data%20scientist
203             director%20%20data%20science
204                  lead%20data%20scientist
205    analytic%20data%20science%20scientist
206                   big%20data%20scientist
207            consulting%20data%20scientist
208                  data%20science%20fellow
209       

# Main code used for the scraping.

In [54]:
########################## SETS UP SOME EMPTY LISTS ####################
location = []
company = []
job_title = []
salary = []
city_search = []
title_search = []
link = []
job_desc = []
date_since_post = []
company_rating = []

### Load count file which stores information on how far through loop python was on last run


########################## FILES ARE OUTPUTTED TO CSVS, 
########################## TRY: JUST CHECKS IF THIS IS THE FIRST TIME THE MODEL IS RUN
########################## -IF THE MODEL HAS BEEN RUN BEFORE THEN IT IMPORTS THE CSVS WHERE NEEDED
########################## EXCEPT: OR WHETHER IT HAS BEEN RUN BEFORE
########################## - IF THE MODEL HASNT BEEN RUN BEFORE IT CREATES NEW EMPTY CSVS TO BE USED LATER IN CODE

try: # try and load previous CSV files 
    counts = pd.read_csv('/Users/stuart/Desktop/GA2/DSI20-lessons/projects/project-4/count_file.csv', sep=',', index_col=False) 
    j_title_count = counts.j_title1[0]
    cities_count = counts.cities_count1[0]
    
    stats = pd.read_csv('/Users/stuart/Desktop/GA2/DSI20-lessons/projects/project-4/stats_file.csv', sep=',', index_col=False) 
    results_unique_count = stats['Total number of unique results so far:'][0]
    results_salaries_count = stats['Total number of salaries collected:'][0]
    total_time = stats['Total time run for:'][0]
    
except: #otherwise just start again 
    j_title_count = 0
    cities_count = 0
    df_empty = pd.DataFrame(list())
    df_empty.to_csv('results.csv')
    df_empty.to_csv('stats_file.csv')
    df_empty.to_csv('count_file.csv')
    results_unique_count = 0
    results_salaries_count = 0
    start = time.time()
    total_time = 0
    
    # Empty pandas DF and load empty to CSV -> THIS IS WHERE THE INFO IS STORED PRIOR TO BEING SAVED TO CSV
    results = pd.DataFrame({'Location': location,
                                              'Company': company,
                                             'Job_Title': job_title, 
                                             'Salary': salary,
                                             'Title_Searched': title_search,
                                             'City_Searched': city_search,
                                             'Link': link,
                                             'Job Description': job_desc,
                                             'Date Since Posted': date_since_post,
                                             'Company Rating': company_rating})
        
    results_unique = results.drop_duplicates(keep='first', ignore_index=True, subset=['Company', 'Job_Title', 'Salary', 'Title_Searched', 'City_Searched'])   
    results_unique.to_csv('results.csv', mode='a', header=True)

########################## BELOW IS THE MAIN BODY OF CODE WHICH ITERATES THROUGH THE WEBPAGES AND SAVES RESULTS.    
########################## THERE ARE 3 LOOPS TO ITERATE THROUGH JOB TITLES, CITY, EACH PAGE.  
########################## AFTER ALL PAGES FOR A CITY HAVE BEEN SEARCHED FOR UNDER 1ST JOB TITLE, THE RESULTS ARE
########################## FILTERED TO REMOVE DUPLICATES, SALARIES COUNTED THEN SAVED TO CSV.
########################## IT THEM MOVES ONTO NEXT CITY. AFTER WHICH RESULTS ARE APPENDED TO ORIGINAL CSV
########################## ONCE ALL CITIES ARE SEARCHED IT MOVES ONTO NEXT JOB TITLE E.G. DATA INTERN AND LOOPS
########################## THROUGH ALL CITIES AGAIN. 
for j_title in job_titles_dscience2: 
#     print('Job title searched:',j_title)
    
    
    for city in tqdm(cities4): # .loc[41747] = chicago
        
        city_searched_percentage = 100*cities_count/cities.shape[0]
 
        pages_to_loop = pages_estimate("https://www.indeed.com/jobs?q=" + j_title + "&l=" + city + "&start=" + '0' + "%22&vjk=b9dedd85514851bc")
#         print('Pages to loop:', pages_to_loop)
#         print(city, pages_to_loop)
        for page in range(0,pages_to_loop,10):
            url = "https://www.indeed.com/jobs?q=" + j_title + "&l=" + city + "&start=" + str(page) + "&vjk=b9dedd85514851bc"
#             print(url)
            r = requests.get(url, proxies=proxies, verify='/Users/stuart/Downloads/zyte-smartproxy-ca.crt')
            soup = BeautifulSoup(r.text, 'html.parser')

            for i in soup.find_all('div', attrs={'class':'slider_container'}):
                try: 
                    location.append(i.find('div', attrs={'class':'companyLocation'}).text)
                except:
                    location.append(np.nan)

                try: 
                    company.append(i.find('span', attrs={'class':'companyName'}).text)
                except:
                    company.append(np.nan)

                try: 
                    job_title.append(i.find('h2', attrs={'class':'jobTitle'}).text)
                except:
                    job_title.append(np.nan)

                try: 
                    salary.append(i.find('div', attrs={'class':'metadata salary-snippet-container'}).text)
                except:
                    salary.append(np.nan)
                    
                try: 
                    job_desc.append(i.find('div', attrs={'class':'job-snippet'}).text)
                except:
                    job_desc.append(np.nan)
                   
                try: 
                    date_since_post.append(i.find('span', attrs={'class':'date'}).text)
                except:
                    date_since_post.append(np.nan)
                
                try: 
                    company_rating.append(i.find('span', attrs={'aria-hidden':'true'}).text)
                except:
                    company_rating.append(np.nan)
                
                
                city_search.append(city)
                title_search.append(j_title)
                link.append(url)      
                
                
        results = pd.DataFrame({'Location': location,
                                             'Company': company,
                                             'Job_Title': job_title, 
                                             'Salary': salary,
                                             'Title_Searched': title_search,
                                             'City_Searched': city_search,
                                             'Link': link,
                                             'Job Description': job_desc,
                                             'Date Since Posted': date_since_post,
                                             'Company Rating': company_rating})
        
        results_unique = results.drop_duplicates(keep='first', ignore_index=True, subset=['Company', 'Job_Title', 'Salary', 'Title_Searched', 'City_Searched'])
        results_unique_count += results_unique.shape[0]
        results_salaries_count += results_unique.Salary.notnull().sum()
        
########################## HERE RESULTS ARE FILTERED TO CHECK FOR UNIQUE VALUES IN THE CITY BEFORE BEING SAVED TO THE CSV
########################## NOTE THIS WAS NOT PERFECT AS IT DOES NOT FILTER OUT REPEAT RESULTS ACCROSS DIFFERENT CITIES
########################## E.G. IF THE SAME JOB CAME UP IN NEWYORK AND LA THE CSV WOULD STILL HAVE BOTH JOBS 
########################## THEREFORE THERE WOULD BE DUPLICATES
########################## THIS WAS CLEANED UP LATER
########################## NOTE THAT THIS WAS TERRIBLY INEFFICIENT AS I SCRAPED 300K URLS, ACCROSS 10% OF US CITIES
########################## AND ONLY GOT 1000K SALARIES.
########################## IT WOULD HAVE BEEN MUCH BETTER TO FOCUS ON THE LARGEST CITIES AND SECONDLY ON USING 
########################## DIFFERENT JOB TITLES

        results_unique.to_csv('results.csv', mode='a', header=False)
        
        location = []
        company = []
        job_title = []
        salary = []
        city_search = []
        title_search = []
        link = []
        job_desc = []
        date_since_post = []
        company_rating = []
        
        end = time.time()
#         total_time += (end - start)
    
        stats = pd.DataFrame({'Total number of unique results so far:': results_unique_count,
                              'Total number of salaries collected:': results_salaries_count,
                              'Percentage of cities searched:':city_searched_percentage,
                              'Total time run for:': total_time }, index = [0])
        stats.to_csv('stats_file.csv')
        
        count_file = pd.DataFrame({'j_title1': j_title_count,
                                             'cities_count1': cities_count}, index=[0])
        count_file.to_csv('count_file.csv')
        cities_count += 1
    j_title_count += 1
    cities_count = 0
    


100%|██████████| 5/5 [14:06<00:00, 169.32s/it]
